In [3]:
import math
from pathlib import Path
from typing import Any
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

import warnings

<img src="images/transformers.png" width=500>

The Transformer architecture has two main blocks: the **encoder** and the **decoder.**

- **Encoder:** It has a Multi-Head Attention mechanism and a fully connected Feed-Forward network. There are also residual connections around the two sub-layers, plus layer normalization for the output of each sub-layer. All sub-layers in the model and the embedding layers produce outputs of dimension $d_{\text{model}} = 512$

- **Decoder:** The decoder follows a similar structure, but it inserts a third sub-layer that performs multi-head attention over the output of the encoder block. There is also a modification of the self-attention sub-layer in the decoder block to avoid positions from attending to subsequent positions. This masking ensures that the predictions for position $i$ depend solely on the known outputs at positions less than $i$.

Both the encoder and decode blocks are repeated $N$ times. In the original paper, they defined $N=6$, and we will define a similar value in this notebook.